In [13]:
# TSP：BerLin52
berlin52 = [[565,575],[25,185],[345,750],[945,685],[845,655],
[880,660],[25,230],[525,1000],[580,1175],[650,1130],[1605,620],
[1220,580],[1465,200],[1530,5],[845,680],[725,370],[145,665],
[415,635],[510,875],[560,365],[300,465],[520,585],[480,415],
[835,625],[975,580],[1215,245],[1320,315],[1250,400],[660,180],
[410,250],[420,555],[575,665],[1150,1160],[700,580],[685,595],
[685,610],[770,610],[795,645],[720,635],[760,650],[475,960],
[95,260],[875,920],[700,500],[555,815],[830,485],[1170,65],
[830,610],[605,625],[595,360],[1340,725],[1740,245]]

In [52]:
# import 
import numpy as np
from random import sample
import random
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import pairwise_distances
import math
import pandas as pd

In [15]:
distances = pairwise_distances(berlin52)
n_city = len(distances)

In [16]:
# algorithm configuration
max_it = 50
num_ants = 30
decay_factor = 0.6
c_heur = 2.5
c_hist = 1.0
# execute the algorithm
# best = search(berlin52, max_it, num_ants, decay_factor, c_heur, c_hist)

In [17]:
# 随机初始解
def get_init_x(n_city = 52):
    return np.random.permutation(n_city)
# 计算适应度函数：
def get_fitness(x):
    global distance
    fitness = sum([distances[x[i]][x[i+1]]  for i in range(len(x)-1)])
    fitness += distances[x[len(x)-1]][x[0]]
    return round(fitness,2)

In [18]:
def select_next_city(choices):
    # 根据累计概率随机选择
    sum_ = 0 
    result = {}
    for prob in choices:
        result[prob['prob']] = result.get(prob['prob'], 0) + prob['prob']
        sum_0 = result[prob['prob']]
    if sum_==0:
        return sample(choices,1)[0]['city']
    sel = random.random()
    for i,v in enumerate(choices):
        v -= (v['prob']/sum_)
        if v <= 0 :
            return choices[i]['city']
    
    return  random.sample(choices)['city']

In [19]:
def calculate_choices(last_city,exclude,pheromone,c_heur,c_hist):
    global distances
    choices = []
    for i in range(n_city):
        if i in exclude:
            continue
        prob = {'city':i}
        prob['history'] = pheromone[last_city][i]** c_hist
        prob['distance'] = distances[last_city][i]
        prob['heuristic'] = (1.0/prob['distance'])**c_heur
        prob['prob'] = prob['history'] * prob['heuristic']
        choices.append(prob)
    return choices

In [31]:
def stepwise_const(phero, c_heur, c_hist):
    # 随机一个初始city_id,不断补全后续city_id
    perm = []
    # 获取随机下标
    perm.append(random.sample(range(n_city),1)[0])
    while len(perm) < n_city:
        choices = calculate_choices(perm[-1],perm,phero,c_heur,c_hist)
        next_city = select_next_city(choices)
        perm.append(next_city)
    return perm

In [54]:
def initialise_pheromone_matrix(n_city,cost):
    pheromone = [ [cost]*n_city  for i in range(n_city)]
    return pheromone

def decay_pheromone(pheromone, decay_factor):
    for i in range(len(pheromone)):
        pheromone[i] = [(1.0- decay_factor)*p for p in pheromone[i] ]


def update_pheromone(pheromone, solutions):
    for sol in solutions:
        for i in range(len(sol['vector'])):
                if i == len(sol['vector'])-1:
                    y = sol['vector'][0]
                else:
                    y = sol['vector'][i+1]
                pheromone[i][y] += (1.0/sol['cost'])
                pheromone[y][i] += (1.0/sol['cost'])
    

In [56]:
def search(n_city, max_it, num_ants, decay_factor, c_heur, c_hist):
    best = {'vector': get_init_x(n_city)}
    best['cost'] = get_fitness(best['vector'])
    # get_pop 根据cost，
    pheromone = initialise_pheromone_matrix(n_city, best['cost'])
    n = 0
    while n< max_it:
        solutions = []
        for i in range(num_ants):
            candidate = {}
            candidate['vector'] = stepwise_const(pheromone, c_heur, c_hist)
            candidate['cost'] = get_fitness(candidate['vector'])
            if candidate['cost'] < best['cost']:
                 best = candidate 
            print(best)
            solutions.append(candidate)
        decay_pheromone(pheromone, decay_factor)
        update_pheromone(pheromone, solutions)
    return best

In [46]:
best = {'vector': get_init_x(n_city)}
best['cost'] = get_fitness(best['vector'])

In [47]:
best

{'vector': array([ 3, 10, 33, 22, 40, 14, 16, 21, 24,  1,  6, 11,  9, 47, 19, 37, 17,
         2, 27, 44, 26, 18, 34,  0, 28, 30, 23,  8, 29, 36, 38, 12, 31, 32,
        51, 50,  5, 48,  7, 49, 39, 42, 43, 20, 45,  4, 35, 13, 15, 41, 25,
        46]), 'cost': 29799.34}

In [36]:
pheromone = initialise_pheromone_matrix(52,40000)

In [28]:
choices = calculate_choices(10,[10],pheromone,c_heur,c_hist)

In [29]:
sample(choices,1)[0]['city']

39

In [27]:
select_next_city(choices)

26